In [1]:
import numpy as np 
import sys
sys.path.append("../")
import pickle as pkl
import os
from pathlib import Path
import fnmatch
from glob import glob
import search_world
import pandas as pd

In [2]:
# reading in scalar and vector data filenames for a specific path
job_id = '21900215'
data_dir = os.path.join('../logs/', job_id)
config_fns = [fname for root, _, _ in os.walk(data_dir) for fname in glob(os.path.join(root, 'config.log'))]
scalar_fns = [fname for root, _, _ in os.walk(data_dir) for fname in glob(os.path.join(root, 'scalar'))]
vector_fns = [fname for root, _, _ in os.walk(data_dir) for fname in glob(os.path.join(root, 'vector'))]

In [3]:
# loading in vector data
vector_arrs = []
for vector_fn in vector_fns[:2]:
    with open(vector_fn, 'rb') as f:
        vector_arrs += pkl.load(f)

In [35]:
# all together now
scalar_arrs = []
vector_dfs = []
scalar_dicts = []
for idx, (vector_fn, scalar_fn) in enumerate(zip(vector_fns[:5], scalar_fns[:5])):
    with open(scalar_fn, 'rb') as f:
        scalar_dict = pkl.load(f)
    scalar_dict = scalar_dict['env'].info()
    scalar_dict.update({'dataset_index': idx})
    scalar_dicts.append(scalar_dict)
    with open(vector_fn, 'rb') as f:
        vector_dicts = pkl.load(f)
        flattened_vector_dicts = []
        
        for d in vector_dicts:
            t = {}
            for k, v in d.items():
                if isinstance(v, dict):
                    for nested_k, nested_v in v.items():        
                        t.update({nested_k: nested_v})
                else:
                    t.update({k: v})      
            flattened_vector_dicts.append(t)
        df = pd.DataFrame(flattened_vector_dicts) 
        df['dataset_index'] = idx
        df['trial_index'] = df.done.cumsum().shift(fill_value=0)
        changes = df.columns.difference(['trial_index', 'dataset_index', 'job_id']).tolist()
        f = lambda x: x.tolist() if len(x) > 1 else x
        df = df.groupby(['job_id', 'dataset_index', 'trial_index'])[changes].agg(f).reset_index()
        df = df[df.apply(lambda x: True in x.done, axis=1)].reset_index()
        df.trial_index += idx
        vector_dfs.append(df)
scalar_df = pd.DataFrame(scalar_dicts)
df = pd.concat(vector_dfs).reset_index()
df = df.merge(scalar_df, on='dataset_index')

In [36]:
# TODO: new column for initial condition

In [47]:
df = df.set_index(['length', 'n_corridors', 'agent_initial_position', 'target_position']) # grouping by initial condition and maze configuration

level_0  index  \
length n_corridors agent_initial_position target_position                   
5      4           12                     3                      0      0   
                                          3                      1      1   
3      4           0                      6                      0      0   
                                          6                      1      1   
                                          6                      2      2   
5      4           21                     9                      0      0   

                                                                                                      job_id  \
length n_corridors agent_initial_position target_position                                                      
5      4           12                     3                length_5_n_corridors_4_agent_initial_position_...   
                                          3                length_5_n_corridors_4_agent_initial_position_...   
3      4           0                      6                length_3_n_corridors_4_agent_initial_position_...   
                                          6                length_3_n_corridors_4_agent_initial_position_...   
                                          6                length_3_n_corridors_4_agent_initial_position_...   
5      4           21                     9                length_5_n_corridors_4_agent_initial_position_...   

                                                           dataset_index  \
length n_corridors agent_initial_position target_position                  
5      4           12                     3                            0   
                                          3                            0   
3      4           0                      6                            1   
                                          6                            1   
                                          6                            1   
5      4           21                     9                            2   

                                                           trial_index  \
length n_corridors agent_initial_position target_position                
5      4           12                     3                          0   
                                          3                          1   
3      4           0                      6                          1   
                                          6                          2   
                                          6                          3   
5      4           21                     9                          2   

                                                                                                      action  \
length n_corridors agent_initial_position target_position                                                      
5      4           12                     3                                [[0, 1], [0, 1], [-1, 0], [0, 0]]   
                                          3                                [[0, 1], [0, 1], [-1, 0], [0, 0]]   
3      4           0                      6                                         [[1, 0], [0, 1], [0, 0]]   
                                          6                                         [[1, 0], [0, 1], [0, 0]]   
                                          6                                         [[1, 0], [0, 1], [0, 0]]   
5      4           21                     9                [[-1, 0], [-1, 0], [0, -1], [0, 0], [0, -1], [...   

                                                                                             agent_position  \
length n_corridors agent_initial_position target_position                                                     
5      4           12                     3                                [[3, 6], [3, 7], [2, 7], [1, 7]]   
                                          3                                [[3, 6], [3, 7], [2, 7], [1, 7]]   
3 

In [11]:
# TODO: new column for maze configuration

In [ ]:
# Need to think of a way to compute pairwise similarity metric